In [ ]:
from pyspark.sql import SparkSession
# Initialize Spark Session


spark = SparkSession.builder \
.appName("Analyzed Plan and  Caching") \
.getOrCreate()

In [1]:
customers_schema = "customer_id INT, name STRING, city STRING, registration_date STRING, is_active BOOLEAN"

customers_df = spark.read \
.format("csv") \
.schema(customers_schema) \
.load("/tmp/customers_500mb.csv")

In [4]:
customers_df.show(5)

+-----------+----------+---------+-----------------+---------+
|customer_id|      name|     city|registration_date|is_active|
+-----------+----------+---------+-----------------+---------+
|       null|      name|     city|            state|     null|
|          0|Customer_0|   Mumbai|        Telangana|     null|
|          1|Customer_1|  Chennai|      West Bengal|     null|
|          2|Customer_2|     Pune|        Karnataka|     null|
|          3|Customer_3|Hyderabad|          Gujarat|     null|
+-----------+----------+---------+-----------------+---------+
only showing top 5 rows



In [5]:
customers_df.show(5)

+-----------+----------+---------+-----------------+---------+
|customer_id|      name|     city|registration_date|is_active|
+-----------+----------+---------+-----------------+---------+
|       null|      name|     city|            state|     null|
|          0|Customer_0|   Mumbai|        Telangana|     null|
|          1|Customer_1|  Chennai|      West Bengal|     null|
|          2|Customer_2|     Pune|        Karnataka|     null|
|          3|Customer_3|Hyderabad|          Gujarat|     null|
+-----------+----------+---------+-----------------+---------+
only showing top 5 rows



In [6]:
customers_df.cache()

DataFrame[customer_id: int, name: string, city: string, registration_date: string, is_active: boolean]

In [7]:
customers_df.show(5)

+-----------+----------+---------+-----------------+---------+
|customer_id|      name|     city|registration_date|is_active|
+-----------+----------+---------+-----------------+---------+
|       null|      name|     city|            state|     null|
|          0|Customer_0|   Mumbai|        Telangana|     null|
|          1|Customer_1|  Chennai|      West Bengal|     null|
|          2|Customer_2|     Pune|        Karnataka|     null|
|          3|Customer_3|Hyderabad|          Gujarat|     null|
+-----------+----------+---------+-----------------+---------+
only showing top 5 rows



In [8]:
customers_df.count()

8767359

In [9]:
customers_df.count()

8767359

In [10]:
customers_df.unpersist()

DataFrame[customer_id: int, name: string, city: string, registration_date: string, is_active: boolean]

In [15]:
customers_df.cache()

DataFrame[customer_id: int, name: string, city: string, registration_date: string, is_active: boolean]

In [16]:
unavailable_df = customers_df.filter(' city= "LA" ')

In [17]:
unavailable_df.show()

+-----------+----+----+-----------------+---------+
|customer_id|name|city|registration_date|is_active|
+-----------+----+----+-----------------+---------+
+-----------+----+----+-----------------+---------+



In [18]:
customers_df.unpersist()

DataFrame[customer_id: int, name: string, city: string, registration_date: string, is_active: boolean]

In [19]:
customers_df.select('customer_id','city').filter('city== "Mumbai"').cache()

DataFrame[customer_id: int, city: string]

In [25]:
customers_df.select('customer_id','city').filter('city== "Mumbai"').explain(mode='extended')

== Parsed Logical Plan ==
'Filter ('city = Mumbai)
+- Project [customer_id#0, city#2]
   +- Relation [customer_id#0,name#1,city#2,registration_date#3,is_active#4] csv

== Analyzed Logical Plan ==
customer_id: int, city: string
Filter (city#2 = Mumbai)
+- Project [customer_id#0, city#2]
   +- Relation [customer_id#0,name#1,city#2,registration_date#3,is_active#4] csv

== Optimized Logical Plan ==
InMemoryRelation [customer_id#0, city#2], StorageLevel(disk, memory, deserialized, 1 replicas)
   +- *(1) Filter (isnotnull(city#2) AND (city#2 = Mumbai))
      +- FileScan csv [customer_id#0,city#2] Batched: false, DataFilters: [isnotnull(city#2), (city#2 = Mumbai)], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://my-cluster-m/tmp/customers_500mb.csv], PartitionFilters: [], PushedFilters: [IsNotNull(city), EqualTo(city,Mumbai)], ReadSchema: struct<customer_id:int,city:string>

== Physical Plan ==
InMemoryTableScan [customer_id#0, city#2]
   +- InMemoryRelation [customer_id#0, city#2], 

In [20]:
customers_df.select('customer_id','city').filter('city== "Mumbai"').show()

+-----------+------+
|customer_id|  city|
+-----------+------+
|          0|Mumbai|
|          4|Mumbai|
|         28|Mumbai|
|         42|Mumbai|
|         47|Mumbai|
|         57|Mumbai|
|         61|Mumbai|
|         75|Mumbai|
|         82|Mumbai|
|         89|Mumbai|
|         90|Mumbai|
|        101|Mumbai|
|        104|Mumbai|
|        108|Mumbai|
|        129|Mumbai|
|        138|Mumbai|
|        142|Mumbai|
|        143|Mumbai|
|        151|Mumbai|
|        158|Mumbai|
+-----------+------+
only showing top 20 rows



In [21]:
customers_df.select('customer_id','city').filter('city== "Mumbai"').count()

1095815

In [22]:
customers_df.select('customer_id','city').filter('city== "Mumbai"').count()

1095815

In [23]:
customers_df.filter('city== "Mumbai"').select('customer_id','city').count()

1095815

In [24]:
customers_df.filter('city== "Mumbai"').select('customer_id','city').count()

1095815

In [26]:
customers_df.filter('city== "Mumbai"').select('customer_id','city').explain(mode='extended')

== Parsed Logical Plan ==
'Project ['customer_id, 'city]
+- Filter (city#2 = Mumbai)
   +- Relation [customer_id#0,name#1,city#2,registration_date#3,is_active#4] csv

== Analyzed Logical Plan ==
customer_id: int, city: string
Project [customer_id#0, city#2]
+- Filter (city#2 = Mumbai)
   +- Relation [customer_id#0,name#1,city#2,registration_date#3,is_active#4] csv

== Optimized Logical Plan ==
Project [customer_id#0, city#2]
+- Filter (isnotnull(city#2) AND (city#2 = Mumbai))
   +- Relation [customer_id#0,name#1,city#2,registration_date#3,is_active#4] csv

== Physical Plan ==
*(1) Filter (isnotnull(city#2) AND (city#2 = Mumbai))
+- FileScan csv [customer_id#0,city#2] Batched: false, DataFilters: [isnotnull(city#2), (city#2 = Mumbai)], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://my-cluster-m/tmp/customers_500mb.csv], PartitionFilters: [], PushedFilters: [IsNotNull(city), EqualTo(city,Mumbai)], ReadSchema: struct<customer_id:int,city:string>



In [27]:
cached_df = customers_df.select('customer_id','city').filter('city== "Mumbai"').cache()

25/02/08 04:51:49 WARN CacheManager: Asked to cache already cached data.


In [28]:
cached_df.show()

+-----------+------+
|customer_id|  city|
+-----------+------+
|          0|Mumbai|
|          4|Mumbai|
|         28|Mumbai|
|         42|Mumbai|
|         47|Mumbai|
|         57|Mumbai|
|         61|Mumbai|
|         75|Mumbai|
|         82|Mumbai|
|         89|Mumbai|
|         90|Mumbai|
|        101|Mumbai|
|        104|Mumbai|
|        108|Mumbai|
|        129|Mumbai|
|        138|Mumbai|
|        142|Mumbai|
|        143|Mumbai|
|        151|Mumbai|
|        158|Mumbai|
+-----------+------+
only showing top 20 rows



In [29]:
cached_df.count()

1095815

In [30]:
cached_df.count()

1095815

In [ ]:
customers_df.select('customer_name').filter('isactive== "True"').cache()

In [ ]:
spark